In [1]:
import requests
from bs4 import BeautifulSoup
import string
import pandas as pd
import json
from datetime import datetime, timedelta
import numpy as np

In [2]:
df = pd.read_csv('C:\studia\informatyka_stosowana\semestr2\magisterka\df_iaaf_top.csv')

In [4]:
df = df.drop('Unnamed: 0', axis = 1)
df

,nazwa,kraj,skrot_kr,numer,data_urodz,url_zaw,pb
0,Usain BOLT,Jamaica,jam,14201847,1986-08-21,https://www.worldathletics.org/athletes/Jamaic...,9.58
1,Tyson GAY,United States,usa,14238588,1982-08-09,https://www.worldathletics.org/athletes/United...,9.69
2,Yohan BLAKE,Jamaica,jam,14201842,1989-12-26,https://www.worldathletics.org/athletes/Jamaic...,9.69
3,Asafa POWELL,Jamaica,jam,14202176,1982-11-23,https://www.worldathletics.org/athletes/Jamaic...,9.72
4,Justin GATLIN,United States,usa,14238562,1982-02-10,https://www.worldathletics.org/athletes/United...,9.74
...,...,...,...,...,...,...,...
9215,Ahmed Hossam EL DIN ALI ABU ZAID,Egypt,egy,14956182,2002-07-29,https://www.worldathletics.org/athletes/Egypt/...,10.55
9216,Akito SUZUKI,Japan,jpn,14951335,2003-08-07,https://www.worldathletics.org/athletes/Japan/...,10.55
9217,Toshiaki ATOMI,Japan,jpn,14963765,1997-01-12,https://www.worldathletics.org/athletes/Japan/...,10.55
9218,Jackson ROWE,Australia,aus,15006642,2003-01-01,https://www.worldathletics.org/athletes/Austra...,10.55


In [7]:
#od 2009, bo do 2008 mamy tylko limit 10.55 czyli taki jak wczesniej
lista_lat =list(range(2009, 2022+1))
lista_lat

[2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

In [9]:
url_lat = []
for i in range(len(lista_lat)):
    url_lat.append('https://www.worldathletics.org/records/toplists/sprints/100-metres/outdoor/men/senior/' + str(lista_lat[i]) + '?regionType=world&timing=electronic&windReading=regular&page=1&bestResultsOnly=true')

In [12]:
soup_lat = []
for i in range(len(url_lat)):
    req = requests.get(url_lat[i]).content
    soup_lat.append(BeautifulSoup(req))

In [21]:
ile_stron_lat = []
for i in range(len(soup_lat)):
    ile_stron_lat.append(int(str(soup_lat[i].find('div', class_ = 'container--pagination__inner').find_all('form')[0]).split('max="')[1].split('"')[0]))

In [22]:
ile_stron_lat   #widac, ze coraz wiecej jedynie w 2020 (35), mniej bo pandemia

[27, 39, 42, 48, 49, 53, 49, 50, 54, 58, 64, 35, 69, 75]

In [35]:
url = {}
for i in range(len(ile_stron_lat)):
    lista_url = []
    for j in range(ile_stron_lat[i]):
        lista_url.append('https://www.worldathletics.org/records/toplists/sprints/100-metres/outdoor/men/senior/' + str(lista_lat[i]) + '?regionType=world&timing=electronic&windReading=regular&page=' + str(j+1) + '&bestResultsOnly=true')
    url[lista_lat[i]] = lista_url
url

{2009: ['https://www.worldathletics.org/records/toplists/sprints/100-metres/outdoor/men/senior/2009?regionType=world&timing=electronic&windReading=regular&page=1&bestResultsOnly=true',
  'https://www.worldathletics.org/records/toplists/sprints/100-metres/outdoor/men/senior/2009?regionType=world&timing=electronic&windReading=regular&page=2&bestResultsOnly=true',
  'https://www.worldathletics.org/records/toplists/sprints/100-metres/outdoor/men/senior/2009?regionType=world&timing=electronic&windReading=regular&page=3&bestResultsOnly=true',
  'https://www.worldathletics.org/records/toplists/sprints/100-metres/outdoor/men/senior/2009?regionType=world&timing=electronic&windReading=regular&page=4&bestResultsOnly=true',
  'https://www.worldathletics.org/records/toplists/sprints/100-metres/outdoor/men/senior/2009?regionType=world&timing=electronic&windReading=regular&page=5&bestResultsOnly=true',
  'https://www.worldathletics.org/records/toplists/sprints/100-metres/outdoor/men/senior/2009?regio

In [70]:
soup = {}
for j in (lista_lat):
    print(j)
    lista_soup = []
    for i in range(len(url[j])):
        req = requests.get(url[j][i]).content
        lista_soup.append(BeautifulSoup(req))
    soup[j] = lista_soup
soup

2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [95]:
strona_z_limit = {}
for j in range(len(soup)):
    for i in range(len(soup[lista_lat[j]])):
        mark = float(soup[lista_lat[j]][i].find('table', class_ = 'records-table').find_all('tr')[1].find_all('td')[1].text.replace(' ', '').replace('\n',''))
        if mark > 10.55:
            strona_z_limit[lista_lat[j]] = i-1
            break
strona_z_limit

{2009: 7,
 2010: 7,
 2011: 7,
 2012: 8,
 2013: 8,
 2014: 9,
 2015: 9,
 2016: 10,
 2017: 10,
 2018: 10,
 2019: 11,
 2020: 5,
 2021: 12,
 2022: 15}

In [156]:
for i in range(7, len(soup[2009])):
    zaw = soup[2009][i].find('table', class_ = 'records-table').find_all('tr')
    for j in range(len(zaw)-1):
        #print(i, j+1, zaw[j+1].find_all('td')[1])
        print(str(zaw[j+1].find_all('td')[3].find_all('a')[0]).split('"'))#[1]#.split('-')[-1].split('=')[-1])
    
    #if str(ile_dan[j]).split('"')[1] == 'DOB':
        

['<a href=', '/athletes/athlete=14223406', '>\n            Anton OLEFIR\n        </a>']
['<a href=', '/athletes/indonesia/avip-dwi-cahyo-014379726', '>\n            Avip DWI CAHYO\n        </a>']
['<a href=', '/athletes/athlete=14212527', '>\n            Idriss Khalid ZOUGARI\n        </a>']
['<a href=', '/athletes/athlete=14216529', '>\n            Patryk ĆWIEK\n        </a>']
['<a href=', '/athletes/athlete=14256057', '>\n            Alvaro CASSIANI\n        </a>']
['<a href=', '/athletes/athlete=14228866', '>\n            Yuan-Kai LIU\n        </a>']
['<a href=', '/athletes/athlete=14197860', '>\n            Arvind ALAGUVEL\n        </a>']
['<a href=', '/athletes/athlete=14177938', '>\n            Marcus THOMPSON\n        </a>']
['<a href=', '/athletes/pr-of-china/huang-wei-014171171', '>\n            Huang WEI\n        </a>']
['<a href=', '/athletes/athlete=14232893', '>\n            Greg BOLDEN\n        </a>']
['<a href=', '/athletes/athlete=14246467', '>\n            Larrone MOOR

['<a href=', '/athletes/athlete=14247769', '>\n            Trinity OTTO\n        </a>']
['<a href=', '/athletes/united-states/geoff-navarro-014335765', '>\n            Geoff NAVARRO\n        </a>']
['<a href=', '/athletes/united-states/antonial-marshall-014366479', '>\n            Antonial MARSHALL\n        </a>']
['<a href=', '/athletes/united-states/isaiah-haywood-014367901', '>\n            Isaiah HAYWOOD\n        </a>']
['<a href=', '/athletes/united-states/damonte-bell-014368197', '>\n            Damonte BELL\n        </a>']
['<a href=', '/athletes/united-states/leonard-steed-014366158', '>\n            Leonard STEED\n        </a>']
['<a href=', '/athletes/united-states/daniel-belch-014232315', '>\n            Daniel BELCH\n        </a>']
['<a href=', '/athletes/united-states/eman-fondren-014237909', '>\n            Eman FONDREN\n        </a>']
['<a href=', '/athletes/united-states/michael-wright-014255586', '>\n            Michael WRIGHT\n        </a>']
['<a href=', '/athletes/un

['<a href=', '/athletes/athlete=14192963', '>\n            Stefan KÖPF\n        </a>']
['<a href=', '/athletes/athlete=14168869', '>\n            Petr HNÍZDIL\n        </a>']
['<a href=', '/athletes/athlete=14197097', '>\n            Dávid ÓNODI\n        </a>']
['<a href=', '/athletes/spain/eduardo-piqueras-014165361', '>\n            Eduardo PIQUERAS\n        </a>']
['<a href=', '/athletes/athlete=14329812', '>\n            El Habib SAINDOU\n        </a>']
['<a href=', '/athletes/france/loic-nannette-014186718', '>\n            Loic NANNETTE\n        </a>']
['<a href=', '/athletes/germany/daniel-kramar-014363606', '>\n            Daniel KRAMAR\n        </a>']
['<a href=', '/athletes/united-states/matt-kerswill-014375104', '>\n            Matt KERSWILL\n        </a>']
['<a href=', '/athletes/athlete=14229315', '>\n            Emrah ALTUNKALEM\n        </a>']
['<a href=', '/athletes/athlete=14375398', '>\n            Adrián BURRIEL\n        </a>']
['<a href=', '/athletes/pr-of-china/xia

In [135]:
len(soup[2009][26].find('table', class_ = 'records-table').find_all('tr'))

83

In [137]:
len(soup[2009][1].find('table', class_ = 'records-table').find_all('tr')))

SyntaxError: unexpected EOF while parsing (<ipython-input-137-ee2344453308>, line 1)